In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import KNNImputer

# Cargar y revisar el dataset

In [ ]:
df = pd.read_excel('./DatasetActivitiesRunning.xlsx', engine="openpyxl")
df.shape

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351195 entries, 0 to 351194
Data columns (total 16 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   UserId                            351195 non-null  int64  
 1   Type                              351195 non-null  object 
 2   Name                              351195 non-null  object 
 3   StartTimeUtc                      351195 non-null  object 
 4   DurationInSeconds                 351195 non-null  int64  
 5   DistanceInMeters                  350216 non-null  float64
 6   Steps                             347187 non-null  float64
 7   AverageSpeedInMetersPerSecond     350201 non-null  float64
 8   AveragePaceInMinutesPerKilometer  350143 non-null  float64
 9   TotalElevationGainInMeters        317879 non-null  float64
 10  TotalElevationLossInMeters        318270 non-null  float64
 11  AverageHeartRateInBeatsPerMinute  337981 non-null  f

,UserId,Type,Name,StartTimeUtc,DurationInSeconds,DistanceInMeters,Steps,AverageSpeedInMetersPerSecond,AveragePaceInMinutesPerKilometer,TotalElevationGainInMeters,TotalElevationLossInMeters,AverageHeartRateInBeatsPerMinute,SourceType,SourceName,Warnings,CreationTime
0,56226720,Running,Avellaneda Carrera,2024-09-30 20:36:15.0000000 +00:00,2308,6050.10,6802.0,2.622,6.356471,127.971985,128.66255,145.0,Garmin,Forerunner 35,NaN,2024-10-01 00:00:32.7421846 +00:00
1,33958672,Running,Malvinas Argentinas Carrera,2024-09-30 23:43:23.0000000 +00:00,232,557.72,600.0,2.399,6.947340,NaN,1.00000,91.0,Garmin,Forerunner 255S Music,DurationLessThanFiveMinutes,2024-10-01 00:00:37.8144259 +00:00
2,33958672,Running,Malvinas Argentinas Carrera,2024-09-30 23:53:23.0000000 +00:00,320,770.48,856.0,2.407,6.924248,3.000000,1.00000,114.0,Garmin,Forerunner 255S Music,NaN,2024-10-01 00:00:52.2962825 +00:00
3,39156304,Running,Córdoba Carrera,2024-09-30 23:26:07.0000000 +00:00,1895,5005.62,5254.0,2.642,6.308352,96.905790,107.28553,146.0,Garmin,Forerunner 45,NaN,2024-10-01 00:01:02.0718947 +00:00
4,10154408,Running,Rosario Carrera,2024-09-30 21:59:39.0000000 +00:00,4386,10376.22,11452.0,2.365,7.047216,71.099960,71.93257,143.0,Garmin,Forerunner 55,NaN,2024-10-01 00:01:30.1037912 +00:00


# Manejo de valores nulos

In [ ]:
# Ver cantidad de valores nulos por columna
print("\n🔹 Valores nulos antes de la limpieza:")
print(df.isnull().sum())

# Imputar valores nulos numéricos con la mediana
df.fillna(df.median(numeric_only=True), inplace=True)

# Imputar valores nulos categoricos con el modo
df.fillna(df.mode().iloc[0], inplace=True)

print("\n🔹 Valores nulos despues de la limpieza:")
print(df.isnull().sum())


🔹 Valores nulos antes de la limpieza:
UserId                       0
Name                         0
StartTimeUtc                 0
DurationInSeconds            0
DistanceInMeters             0
                            ..
SourceName_vívomove Trend    0
SourceName_vívosmart 4       0
SourceName_vívosmart HR      0
SourceName_vívosmart HR+     0
SourceName_vívosport         0
Length: 257, dtype: int64

🔹 Valores nulos despues de la limpieza:
UserId                       0
Name                         0
StartTimeUtc                 0
DurationInSeconds            0
DistanceInMeters             0
                            ..
SourceName_vívomove Trend    0
SourceName_vívosmart 4       0
SourceName_vívosmart HR      0
SourceName_vívosmart HR+     0
SourceName_vívosport         0
Length: 257, dtype: int64


##Eliminamos la columnas

In [ ]:
df.drop(columns=["Warnings"], inplace=True)
df.drop(columns=["Type"], inplace=True)

#Codificación de variables categóricas


Revisamos qué variables son categóricas y las transformamos.


In [ ]:
cat_cols = ["Type", "SourceType", "SourceName"]

# Aplicar One-Hot Encoding
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

print("\n🔹 Estado del dataset después de codificación de variables categóricas:")
print(df.head())
df.head()

KeyError: "None of [Index(['Type', 'SourceType', 'SourceName'], dtype='object')] are in the [columns]"

# Escalado y Normalización

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

num_cols = ["DurationInSeconds", "DistanceInMeters", "Steps", "AverageSpeedInMetersPerSecond", "AveragePaceInMinutesPerKilometer"]
df[num_cols] = scaler.fit_transform(df[num_cols])

# Ver primeras filas después de escalado
print("\n🔹 Estado del dataset después de escalado:")
print(df.head())


🔹 Estado del dataset después de escalado:
     UserId                         Name                        StartTimeUtc  \
0  56226720           Avellaneda Carrera  2024-09-30 20:36:15.0000000 +00:00   
1  33958672  Malvinas Argentinas Carrera  2024-09-30 23:43:23.0000000 +00:00   
2  33958672  Malvinas Argentinas Carrera  2024-09-30 23:53:23.0000000 +00:00   
3  39156304              Córdoba Carrera  2024-09-30 23:26:07.0000000 +00:00   
4  10154408              Rosario Carrera  2024-09-30 21:59:39.0000000 +00:00   

   DurationInSeconds  DistanceInMeters     Steps  \
0           0.688380          0.005267  0.030865   
1           0.687571          0.000485  0.002723   
2           0.687605          0.000671  0.003884   
3           0.688219          0.004357  0.023841   
4           0.689190          0.009033  0.051965   

   AverageSpeedInMetersPerSecond  AveragePaceInMinutesPerKilometer  \
0                       0.002628                          0.000030   
1                      

# Eliminación de outliers (Método IQR)

In [ ]:
Q1 = df[num_cols].quantile(0.25)
Q3 = df[num_cols].quantile(0.75)
IQR = Q3 - Q1

# Filtrar datos dentro del rango IQR
df = df[~((df[num_cols] < (Q1 - 1.5 * IQR)) | (df[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

# Ver información después de eliminar outliers
print("\n🔹 Estado del dataset después de eliminar outliers:")
print(df.describe())


🔹 Estado del dataset después de eliminar outliers:
             UserId  DurationInSeconds  DistanceInMeters          Steps  \
count  2.976890e+05      297689.000000      2.976890e+05  297689.000000   
mean   4.695699e+07           0.688500      6.298986e-03       0.031272   
std    2.710695e+07           0.000601      3.758220e-03       0.019995   
min    2.024000e+03           0.687481      5.223000e-08       0.000000   
25%    2.578981e+07           0.688046      3.496529e-03       0.015637   
50%    4.436203e+07           0.688476      6.097235e-03       0.030302   
75%    7.106264e+07           0.688886      8.724822e-03       0.044533   
max    9.799196e+07           0.690372      1.764020e-02       0.094302   

       AverageSpeedInMetersPerSecond  AveragePaceInMinutesPerKilometer  \
count                  297689.000000                     297689.000000   
mean                        0.002814                          0.000029   
std                         0.000514              

# División en conjuntos de entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split

# Definir variables independientes (X) y dependientes (y)
X = df.drop(columns=["UserId"])  # Eliminamos UserId porque no aporta información útil
y = df["DurationInSeconds"]  # Supongamos que DurationInSeconds es la variable objetivo

# División 80% entrenamiento - 20% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar tamaños de los conjuntos
print("\n🔹 Tamaño de los conjuntos después de la división:")
print(f"Entrenamiento: {X_train.shape}, Prueba: {X_test.shape}")


🔹 Tamaño de los conjuntos después de la división:
Entrenamiento: (238151, 256), Prueba: (59538, 256)
